### Importing Modules and Loading Data

In [1]:
# Import needed modules 
import pandas as pd
import sqlite3

In [2]:
!pip install ipython-sql

In [4]:
# Load CSV files
df_schools = pd.read_csv(r'C:\Users\kosch\Downloads\ChicagoPublicSchools.csv')

In [5]:
df_crime = pd.read_csv(r'C:\Users\kosch\Downloads\ChicagoCrimeData (1).csv')

In [6]:
df_census = pd.read_csv(r'C:\Users\kosch\Downloads\ChicagoCensusData (1).csv')

In [7]:
# Create a, sqlite3 database
cxn = sqlite3.connect('chicago.db')

In [9]:
# Load files into database
df_schools.to_sql('Schools', cxn, if_exists='replace', index=False)

566

In [10]:
df_census.to_sql('Census', cxn, if_exists='replace', index=False)

78

In [11]:
df_crime.to_sql('Crime', cxn, if_exists='replace', index=False)

533

### Data Analysis using SQLite

In [12]:
%load_ext sql

In [13]:
%sql sqlite:///chicago.db

##### Q1: Find the total number of crimes recorded in the CRIME table.

In [23]:
%%sql
SELECT COUNT(ID) 
FROM Crime

 * sqlite:///chicago.db
Done.


COUNT(ID)
533


##### Q2: List community areas with per capita income less than 11000.

In [24]:
%%sql
SELECT COMMUNITY_AREA_NAME 
FROM Census 
WHERE PER_CAPITA_INCOME < 11000

 * sqlite:///chicago.db
Done.


COMMUNITY_AREA_NAME
West Garfield Park
South Lawndale
Fuller Park
Riverdale


##### Q3: List all case numbers for crimes  involving minors?(children are not considered minors for the purposes of crime analysis)

In [25]:
%%sql
SELECT CASE_NUMBER 
FROM Crime 
WHERE DESCRIPTION LIKE '%MINOR%'

 * sqlite:///chicago.db
Done.


CASE_NUMBER
HL266884
HK238408


##### Q4: List all kidnapping crimes involving a child?

In [26]:
%%sql
SELECT *
FROM Crime
WHERE PRIMARY_TYPE = 'KIDNAPPING'

 * sqlite:///chicago.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,1/26/2007,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


##### Q5: What kinds of crimes were recorded at schools?

In [27]:
%%sql
SELECT DISTINCT PRIMARY_TYPE
FROM Crime
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%' OR '%COLLEGE/UNIVERSITY%'

 * sqlite:///chicago.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


##### Q6: List the average safety score for each type of school.

In [31]:
%%sql
SELECT "Elementary, Middle, or High School" AS school_type, AVG(SAFETY_SCORE) AS AVG_Score
FROM Schools
WHERE SAFETY_SCORE IS NOT NULL
GROUP BY "Elementary, Middle, or High School"

 * sqlite:///chicago.db
Done.


school_type,AVG_Score
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


##### Q7: List 5 community areas with highest % of households below poverty line# 

In [32]:
%%sql
SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY
FROM Census
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
LIMIT 5

 * sqlite:///chicago.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


##### Q8: Which community area is most crime prone?

In [33]:
%%sql
SELECT COMMUNITY_AREA_NUMBER, COUNT(COMMUNITY_AREA_NUMBER) AS Occurances
FROM Crime
WHERE COMMUNITY_AREA_NUMBER IS NOT NULL
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC
LIMIT 1

 * sqlite:///chicago.db
Done.


COMMUNITY_AREA_NUMBER,Occurances
25.0,43


##### Q9: Use a sub-query to find the name of the community area with highest hardship index

In [34]:
%%sql
SELECT COMMUNITY_AREA_NAME
FROM Census
WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) 
                        FROM CENSUS)

 * sqlite:///chicago.db
Done.


COMMUNITY_AREA_NAME
Riverdale


##### Q10: Use a sub-query to determine the Community Area Name with most number of crimes?

In [36]:
%%sql
SELECT COMMUNITY_AREA_NAME
FROM Census
WHERE COMMUNITY_AREA_NUMBER IN
    (SELECT COMMUNITY_AREA_NUMBER
     FROM Crime
     WHERE COMMUNITY_AREA_NUMBER IS NOT NULL
     GROUP BY COMMUNITY_AREA_NUMBER
     ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC
     LIMIT 1)

 * sqlite:///chicago.db
Done.


COMMUNITY_AREA_NAME
Austin
